# Exploring the NIFTY 50 index under different SMA regimes

Markets tend to be manic depressive, going from excessive optimism to excessive pessimism. The distribution of returns tend to be different based on the "regime." A simple way to demarcate these regimes is by using an SMA (Simple Moving Average) to bucket returns into binary groups. Here, we explore how returns are distributed in these buckets.

In [1]:
library(tidyverse)
library(ggthemes)
library(odbc)
library(RPostgres)
library(plutoR)
library(quantmod)
library(PerformanceAnalytics)
library(reshape2)
library(ggrepel)
library(lubridate)

options("scipen"=999)
options(stringsAsFactors = FALSE)
options(repr.plot.width=16, repr.plot.height=8)

source("config.R")
source("goofy/plot.common.R")
source("goofy/misc.common.R")

#initialize
indices<-Indices()

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.2.0     ✔ purrr   0.3.2
✔ tibble  2.1.3     ✔ dplyr   0.8.3
✔ tidyr   0.8.3     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.4.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
Loading required package: xts
Loading required package: zoo

Attaching package: ‘zoo’

The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric

Registered S3 method overwritten by 'xts':
  method     from
  as.zoo.xts zoo 

Attaching package: ‘xts’

The following objects are masked from ‘package:dplyr’:

    first, last

Loading required package: TTR
Registered S3 method overwritten by 'quantmod':
  method            from
  as.zoo.data.frame zoo 
Version 0.4-0 included new data defaults. See ?getSymbols.

Attaching package: ‘PerformanceAnalytics’

The following object is masked from

In [2]:
smaLb <- 50 #number of days
px <- indices$NseTimeSeries() %>%
    filter(NAME == 'NIFTY 50 TR') %>%
    select(TIME_STAMP, CLOSE) %>%
    collect()

#if we break by years, make sure that they have at least 200 days
numDays <- px %>%
    group_by(Y = year(TIME_STAMP)) %>%
    summarize(N = n())

yrs <- numDays %>%
    filter(N >= 200) %>%
    pull(Y)

In [3]:
pXts <- xts(px$CLOSE, px$TIME_STAMP)
names(pXts) <- c('INDEX')
pXts$SMA <- SMA(pXts[,1], smaLb)
pXts$D_RET <- dailyReturn(pXts$INDEX)

In [4]:
statByYear <- data.frame(Y = 0, N_ABOVE = 0, N_BELOW = 0, AVG_ABOVE = 0.0, AVG_BELOW = 0.0)

for(yr in yrs){
    yrStr <- toString(yr)
    numAbove <- as.numeric(sum(ifelse(pXts[yrStr,'INDEX'] > pXts[yrStr,'SMA'], 1, 0)))
    numBelow <- as.numeric(sum(ifelse(pXts[yrStr,'INDEX'] <= pXts[yrStr,'SMA'], 1, 0)))
    
    avgAbove <- as.numeric(mean(pXts[pXts[yrStr,'INDEX'] > pXts[yrStr,'SMA'], 'D_RET']))
    avgBelow <- as.numeric(mean(pXts[pXts[yrStr,'INDEX'] <= pXts[yrStr,'SMA'], 'D_RET']))
    
    statByYear <- rbind(statByYear, c(yr, numAbove, numBelow, avgAbove, avgBelow))
}

statByYear <- statByYear[-1,]
print(statByYear)

      Y N_ABOVE N_BELOW    AVG_ABOVE      AVG_BELOW
2  2000     113     137 0.0040273740 -0.00140179908
3  2001      94     154 0.0028798902  0.00002901356
4  2002     103     148 0.0023112970  0.00024482659
5  2003     173      81 0.0015789966  0.00022844521
6  2004     170      84 0.0002496763  0.00296697022
7  2005     188      63 0.0014948321 -0.00010683350
8  2006     202      48 0.0004034361  0.00378234804
9  2007     203      46 0.0007190228  0.00250554083
10 2008      75     171 0.0012547134  0.00107003596
11 2009     187      56 0.0011811708  0.00102273862
12 2010     168      84 0.0008758752  0.00169469590
13 2011      78     169 0.0034117546  0.00011744337
14 2012     178      73 0.0009465987  0.00144936099
15 2013     151      99 0.0002199312  0.00232158762
16 2014     200      44 0.0004130122  0.00477420691
17 2015     117     131 0.0009171735  0.00128143867
18 2016     147     100 0.0006171436  0.00195244687
19 2017     228      20 0.0007567307  0.00513215838
20 2018     

This notebook was created using [pluto](http://pluto.studio). Learn more [here](https://github.com/shyams80/pluto)